In [ ]:
# @title Scraping Data Live Chat via YT-DLP
# Langkah 1: Install Library
import os
import json
import pandas as pd
import subprocess

# Install yt-dlp jika belum ada
print("Sedang menginstall alat scraping terbaru (yt-dlp)...")
subprocess.run("pip install yt-dlp", shell=True)

# --- KONFIGURASI LINK ---
URL_VIDEO = "https://www.youtube.com/live/9MUY_m7PyQQ?si=sWK1YG7m0BwKGmG_"
NAMA_OUTPUT = "chat"
# ------------------------

def download_and_process_chat(url, filename_base):
    print(f"\n1. Memulai download chat dari: {url}")
    print("   Proses ini menggunakan jalur resmi subtitle YouTube.")
    print("   Mohon tunggu, ini mungkin memakan waktu 1-3 menit...")

    # Hapus file lama jika ada
    if os.path.exists(f"{filename_base}.live_chat.json"):
        os.remove(f"{filename_base}.live_chat.json")

    # Jalankan perintah yt-dlp lewat terminal sistem
    # --write-subs: Ambil subtitle/chat
    cmd = f'yt-dlp --skip-download --write-subs --sub-langs live_chat --output "{filename_base}" "{url}"'
    subprocess.run(cmd, shell=True)

    json_file = f"{filename_base}.live_chat.json"

    if not os.path.exists(json_file):
        print("❌ GAGAL: File chat tidak ditemukan. Mungkin video tidak memiliki replay chat.")
        return

    print("\n2. Download selesai! Sedang mengubah data ke format Excel/CSV...")

    data_bersih = []

    # Buka file JSON hasil download line-by-line
    with open(json_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line)

                # Navigasi struktur data YouTube yang rumit
                # Kita cari "replayChatItemAction" -> "actions" -> "addChatItemAction"
                actions = data.get('replayChatItemAction', {}).get('actions', [])
                for action in actions:
                    item = action.get('addChatItemAction', {}).get('item', {})

                    # Kita hanya ambil Chat Biasa (text), bukan SuperChat/Sticker
                    if 'liveChatTextMessageRenderer' in item:
                        renderer = item['liveChatTextMessageRenderer']

                        # Ambil Pesan
                        pesan_parts = renderer.get('message', {}).get('runs', [])
                        pesan_text = "".join([part.get('text', '') for part in pesan_parts])

                        # Ambil Nama & Waktu
                        nama = renderer.get('authorName', {}).get('simpleText', 'Anonim')
                        waktu = renderer.get('timestampText', {}).get('simpleText', '')

                        data_bersih.append({
                            'Waktu': waktu,
                            'User': nama,
                            'Pesan': pesan_text
                        })

            except Exception:
                continue

    # Simpan ke CSV
    if len(data_bersih) > 0:
        df = pd.DataFrame(data_bersih)
        csv_filename = f"{filename_base}.csv"
        df.to_csv(csv_filename, index=False)

        print(f"\n✅ SUKSES BESAR!")
        print(f"   Berhasil mengambil: {len(df)} pesan chat.")
        print(f"   File tersimpan sebagai: {csv_filename}")
        print("   Silakan download lewat menu folder di kiri.")
        print("-" * 30)
        print(df.head()) # Tampilkan 5 data pertama
    else:
        print("⚠️ Data kosong. Struktur JSON mungkin berbeda atau tidak ada chat teks.")

# Jalankan Fungsi
download_and_process_chat(URL_VIDEO, NAMA_OUTPUT)

Sedang menginstall alat scraping terbaru (yt-dlp)...

1. Memulai download chat dari: https://www.youtube.com/live/9MUY_m7PyQQ?si=sWK1YG7m0BwKGmG_
   Proses ini menggunakan jalur resmi subtitle YouTube.
   Mohon tunggu, ini mungkin memakan waktu 1-3 menit...

2. Download selesai! Sedang mengubah data ke format Excel/CSV...

✅ SUKSES BESAR!
   Berhasil mengambil: 106748 pesan chat.
   File tersimpan sebagai: chat.csv
   Silakan download lewat menu folder di kiri.
------------------------------
   Waktu                    User  \
0  -0:24                @scp0110   
1  -0:23           @RenoReno-l1j   
2  -0:23           @nabiltzy2149   
3  -0:21  @edwardpattinsonoh9778   
4  -0:21         @wekomentar3388   

                                               Pesan  
0                                final onic 2 geek 4  
1                                          kocak mpl  
2                                            mana ni  
3                                          lama kali  
4  lagi nga